In [1]:
#Importing Data from the CSV file
%matplotlib inline
import string
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
#class_0 = number of classes with label = 0
#class_4 = number of classes with label = 4

class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","')
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets (TEST DATA)
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))
    
print("The length of the training set is = ", len(X_train))
print("The number of classes (label=4) =", class_4)
print("The number of classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m

The length of the training set is =  1600000
The number of classes (label=4) = 800000
The number of classes (label=0) = 800000
The length of the test set is =  359


In [2]:
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile, chi2


In [3]:
#tweets_train_pd = pd.read_pickle('tweet_train_lemma.pkl')
#tweets_test_pd = pd.read_pickle('tweet_test_lemma.pkl')
train_actual_classes = pd.read_pickle('train_class.pkl')
test_actual_classes = pd.read_pickle('test_class.pkl')

In [4]:
tweets_train_pd = pd.Series(tweets_train)
tweets_train_pd = tweets_train_pd.apply(lambda x: x.split("|")[1])
tweets_test_pd = pd.Series(tweets_test)
tweets_test_pd = tweets_test_pd.apply(lambda x: x.split("|")[1])

In [5]:
train_classes = train_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
test_classes = test_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
all_classes = train_classes.unique()

In [6]:
def create_minibatch(tweets, classes, batch_size):
    l = len(tweets)
    for i in range(0, l, batch_size):
        X_batch=tweets[i:min(i + batch_size, l)]
        Y_batch=classes[i:min(i + batch_size, l)]
        yield X_batch,Y_batch

In [7]:
model_gauss = GaussianNB([phi_0, phi_4])

In [9]:
#Create minibatches

count_vec = CountVectorizer(analyzer="word", stop_words='english')
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
#X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train_pd))
tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train_pd)))

start=time.time()
for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):

    count_batch=count_vec.transform(X_batch)
    tfidf_train_g=tfidf_transform.transform(count_batch)
    model_gauss.partial_fit(tfidf_train_g.toarray(), Y_batch, classes=all_classes)

print("The time taken for training on batch size=100 using tfidf=", time.time() - start)    

The time taken for training on batch size=100 using tfidf= 8480.386967897415


In [19]:
X_test = tfidf_transform.transform(count_vec.transform(tweets_test_pd))
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

In [20]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=100 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=100 is = 51.25348189415042%


In [10]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=1000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=1000 is = 51.25348189415042%


In [8]:
tweets_shuffled = pd.concat([tweets_train_pd, train_classes], axis=1).sample(frac=1)

In [9]:
tweets_train = list(tweets_shuffled[0])
train_classes = list(tweets_shuffled[1])

In [ ]:
tfidf_vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, min_df=0.001)
tfidf_train = tfidf_vec.fit_transform(tweets_train)

In [14]:
tfidf_train

<1600000x937 sparse matrix of type '<class 'numpy.float64'>'
	with 6503097 stored elements in Compressed Sparse Row format>

In [ ]:
model_gauss.fit(tfidf_train.toarray(), train_classes)

In [ ]:
X_test = tfidf_vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)
print(test_accuracy) #min-df=0.001

In [10]:
count_vec = CountVectorizer(analyzer="word", stop_words='english', min_df=0.001)
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train))
#tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train)))
model_gauss.fit(X_count.toarray(), train_classes)
#Also gives 77.71% accuracy

GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
#Create minibatches

count_vec = CountVectorizer(analyzer="word", stop_words='english', min_df=0.0001)
tfidf_vec = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=True)
#X_count = tfidf_vec.fit_transform(count_vec.fit_transform(tweets_train_pd))
tfidf_transform = (tfidf_vec.fit(count_vec.fit_transform(tweets_train)))
num_batch=0
start=time.time()
for i in range(10):
    for (X_batch, Y_batch) in create_minibatch(tweets_train, train_classes, 1000):
        count_batch=count_vec.transform(X_batch)
        tfidf_train_g=tfidf_transform.transform(count_batch)
        if (num_batch>0): print(model_gauss.score(tfidf_train_g.toarray(), Y_batch))
        model_gauss.partial_fit(tfidf_train_g.toarray(), Y_batch, classes=all_classes)
        num_batch+=1
        
print("The time taken for training on batch size=1000 using tfidf=", time.time() - start) 

0.727
0.697
0.707
0.708
0.703
0.734
0.69
0.737
0.708
0.709
0.708
0.708
0.707
0.712
0.701
0.708
0.708
0.698
0.699
0.72
0.698
0.677
0.694
0.716
0.713
0.727
0.703
0.708
0.693
0.742
0.697
0.693
0.707
0.744
0.712
0.703
0.705
0.678
0.702
0.705
0.682
0.696
0.721
0.711
0.729
0.697
0.696
0.693
0.692
0.717
0.704
0.7
0.706
0.705
0.716
0.69
0.721
0.719
0.706
0.699
0.731
0.7
0.704
0.717
0.703
0.689
0.701
0.7
0.689
0.718
0.674
0.709
0.668
0.71
0.702
0.705
0.708
0.743
0.684
0.7
0.713
0.7
0.685
0.719
0.68
0.673
0.699
0.697
0.704
0.705
0.669
0.714
0.71
0.682
0.707
0.735
0.688
0.698
0.701
0.693
0.703
0.681
0.672
0.699
0.706
0.688
0.716
0.723
0.704
0.717
0.706
0.686
0.707
0.722
0.722
0.691
0.693
0.732
0.705
0.676
0.708
0.7
0.7
0.723
0.718
0.719
0.697
0.716
0.678
0.695
0.704
0.695
0.717
0.738
0.715
0.657
0.693
0.707
0.674
0.692
0.73
0.696
0.7
0.705
0.713
0.707
0.679
0.723
0.726
0.701
0.729
0.7
0.713
0.684
0.689
0.721
0.727
0.715
0.714
0.693
0.719
0.706
0.722
0.705
0.732
0.72
0.704
0.716
0.705
0.709
0.71
0

0.722
0.691
0.702
0.715
0.696
0.713
0.691
0.709
0.684
0.677
0.702
0.729
0.718
0.698
0.708
0.744
0.707
0.687
0.696
0.73
0.71
0.677
0.709
0.711
0.694
0.7
0.721
0.699
0.702
0.723
0.702
0.671
0.697
0.704
0.7
0.714
0.676
0.683
0.703
0.711
0.709
0.708
0.724
0.695
0.714
0.708
0.711
0.706
0.707
0.708
0.693
0.745
0.713
0.679
0.72
0.698
0.693
0.73
0.7
0.695
0.674
0.71
0.703
0.697
0.71
0.704
0.711
0.704
0.684
0.708
0.703
0.683
0.708
0.7
0.702
0.685
0.713
0.727
0.711
0.662
0.68
0.736
0.711
0.691
0.687
0.729
0.708
0.695
0.71
0.699
0.688
0.708
0.698
0.688
0.716
0.709
0.701
0.703
0.683
0.736
0.7
0.724
0.695
0.705
0.71
0.7
0.691
0.705
0.693
0.701
0.703
0.718
0.727
0.701
0.695
0.69
0.697
0.72
0.688
0.71
0.709
0.71
0.691
0.672
0.703
0.711
0.704
0.736
0.708
0.718
0.709
0.716
0.721
0.678
0.696
0.688
0.702
0.692
0.7
0.714
0.686
0.698
0.681
0.733
0.712
0.69
0.714
0.693
0.711
0.705
0.677
0.699
0.68
0.694
0.698
0.718
0.676
0.7
0.72
0.698
0.723
0.678
0.717
0.721
0.706
0.694
0.683
0.701
0.72
0.696
0.699
0.719
0

0.697
0.709
0.677
0.689
0.711
0.695
0.693
0.706
0.719
0.702
0.7
0.691
0.732
0.683
0.712
0.696
0.685
0.696
0.676
0.702
0.717
0.726
0.726
0.703
0.712
0.689
0.719
0.697
0.698
0.703
0.697
0.7
0.704
0.715
0.698
0.701
0.719
0.709
0.686
0.719
0.692
0.707
0.694
0.685
0.724
0.687
0.711
0.708
0.712
0.699
0.703
0.693
0.706
0.707
0.706
0.698
0.708
0.721
0.71
0.702
0.686
0.698
0.73
0.698
0.671
0.726
0.714
0.718
0.693
0.7
0.681
0.719
0.685
0.695
0.712
0.709
0.704
0.693
0.692
0.706
0.696
0.691
0.7
0.718
0.682
0.699
0.685
0.721
0.702
0.722
0.707
0.692
0.693
0.683
0.684
0.69
0.735
0.698
0.724
0.717
0.694
0.702
0.696
0.719
0.71
0.686
0.717
0.708
0.694
0.689
0.712
0.703
0.743
0.689
0.713
0.712
0.699
0.676
0.685
0.712
0.723
0.687
0.689
0.693
0.684
0.736
0.705
0.682
0.702
0.721
0.708
0.673
0.688
0.73
0.698
0.702
0.672
0.689
0.714
0.696
0.69
0.716
0.705
0.709
0.689
0.703
0.724
0.702
0.699
0.708
0.696
0.715
0.702
0.715
0.708
0.716
0.7
0.712
0.694
0.691
0.7
0.742
0.707
0.686
0.703
0.718
0.686
0.713
0.69
0.71


0.713
0.686
0.692
0.741
0.7
0.714
0.684
0.673
0.713
0.706
0.717
0.681
0.698
0.725
0.729
0.739
0.701
0.694
0.679
0.693
0.712
0.706
0.712
0.687
0.67
0.715
0.704
0.695
0.692
0.704
0.699
0.677
0.691
0.689
0.732
0.696
0.698
0.69
0.704
0.723
0.692
0.719
0.695
0.707
0.688
0.719
0.694
0.715
0.711
0.715
0.705
0.714
0.718
0.71
0.714
0.694
0.688
0.714
0.714
0.693
0.709
0.695
0.683
0.694
0.682
0.681
0.701
0.71
0.727
0.708
0.698
0.688
0.719
0.687
0.695
0.707
0.683
0.677
0.696
0.687
0.721
0.702
0.722
0.686
0.71
0.678
0.711
0.71
0.698
0.703
0.693
0.71
0.696
0.702
0.728
0.713
0.686
0.736
0.703
0.699
0.684
0.694
0.693
0.712
0.686
0.683
0.697
0.695
0.709
0.717
0.72
0.702
0.691
0.703
0.701
0.712
0.673
0.709
0.718
0.658
0.699
0.71
0.703
0.691
0.723
0.699
0.715
0.71
0.713
0.722
0.683
0.71
0.717
0.728
0.742
0.706
0.71
0.695
0.705
0.695
0.708
0.719
0.695
0.694
0.677
0.711
0.717
0.696
0.717
0.708
0.703
0.687
0.715
0.679
0.693
0.721
0.713
0.674
0.698
0.723
0.683
0.709
0.71
0.714
0.695
0.711
0.706
0.694
0.715
0

0.697
0.705
0.711
0.715
0.703
0.698
0.703
0.721
0.713
0.711
0.717
0.715
0.724
0.736
0.723
0.723
0.706
0.705
0.693
0.687
0.709
0.698
0.717
0.719
0.698
0.702
0.716
0.677
0.716
0.701
0.715
0.733
0.704
0.706
0.707
0.689
0.754
0.7
0.706
0.717
0.705
0.692
0.69
0.708
0.692
0.722
0.711
0.699
0.698
0.705
0.695
0.692
0.703
0.712
0.691
0.72
0.719
0.697
0.712
0.732
0.694
0.724
0.717
0.688
0.691
0.697
0.712
0.725
0.713
0.699
0.708
0.705
0.666
0.679
0.714
0.712
0.702
0.708
0.743
0.7
0.719
0.697
0.712
0.691
0.722
0.7
0.728
0.705
0.694
0.667
0.688
0.716
0.701
0.677
0.681
0.691
0.739
0.69
0.721
0.717
0.706
0.699
0.703
0.727
0.676
0.712
0.704
0.698
0.71
0.693
0.698
0.696
0.695
0.713
0.697
0.688
0.689
0.717
0.691
0.719
0.705
0.704
0.714
0.703
0.71
0.684
0.739
0.721
0.713
0.694
0.692
0.723
0.707
0.707
0.693
0.709
0.719
0.72
0.714
0.712
0.709
0.701
0.703
0.716
0.713
0.721
0.711
0.699
0.683
0.723
0.698
0.711
0.694
0.711
0.699
0.694
0.688
0.709
0.707
0.682
0.719
0.715
0.699
0.685
0.713
0.685
0.708
0.703
0.71

...

0.697
0.699
0.723
0.679
0.703
0.721
0.701
0.727
0.679
0.718
0.719
0.708
0.695
0.683
0.701
0.722
0.697
0.699
0.719
0.698
0.713
0.705
0.729
0.718
0.709
0.698
0.695
0.674
0.708
0.695
0.693
0.681
0.715
0.708
0.69
0.707
0.717
0.711
0.722
0.679
0.715
0.719
0.707
0.698
0.667
0.726
0.687
0.708
0.735
0.707
0.71
0.761
0.707
0.727
0.697
0.707
0.709
0.706
0.733
0.69
0.738
0.707
0.709
0.71
0.708
0.704
0.713
0.699
0.709
0.71
0.697
0.702
0.722
0.699
0.677
0.694
0.715
0.713
0.726
0.7
0.71
0.691
0.743
0.698
0.691
0.708
0.746
0.714
0.702
0.705
0.678
0.703
0.704
0.684
0.698
0.722
0.705
0.726
0.699
0.696
0.694
0.692
0.714
0.7
0.696
0.706
0.704
0.715
0.69
0.725
0.718
0.702
0.701
0.732
0.7
0.701
0.72
0.706
0.692
0.701
0.697
0.684
0.72
0.676
0.711
0.668
0.711
0.7
0.707
0.705
0.744
0.685
0.699
0.713
0.702
0.688
0.719
0.682
0.676
0.701
0.698
0.704
0.702
0.669
0.711
0.712
0.683
0.704
0.733
0.688
0.701
0.695
0.693
0.707
0.685
0.67
0.702
0.71
0.685
0.713
0.721
0.704
0.717
0.702
0.686
0.713
0.722
0.719
0.696
0.695

0.694
0.715
0.703
0.713
0.708
0.716
0.703
0.71
0.694
0.692
0.7
0.746
0.71
0.687
0.701
0.717
0.689
0.715
0.693
0.713
0.719
0.659
0.693
0.696
0.697
0.694
0.688
0.695
0.688
0.716
0.712
0.709
0.715
0.708
0.695
0.695
0.69
0.706
0.691
0.701
0.678
0.689
0.703
0.704
0.694
0.722
0.711
0.695
0.702
0.726
0.703
0.725
0.711
0.716
0.723
0.694
0.706
0.718
0.698
0.712
0.69
0.71
0.684
0.677
0.699
0.728
0.72
0.7
0.709
0.745
0.708
0.688
0.694
0.729
0.714
0.677
0.709
0.714
0.694
0.699
0.725
0.697
0.7
0.724
0.705
0.673
0.7
0.706
0.701
0.712
0.678
0.688
0.706
0.713
0.712
0.711
0.724
0.697
0.716
0.707
0.714
0.709
0.705
0.711
0.696
0.746
0.715
0.679
0.721
0.7
0.695
0.728
0.7
0.698
0.678
0.709
0.71
0.695
0.711
0.707
0.711
0.703
0.684
0.708
0.705
0.684
0.708
0.702
0.702
0.688
0.715
0.728
0.709
0.665
0.68
0.737
0.714
0.697
0.688
0.727
0.707
0.694
0.709
0.703
0.689
0.709
0.698
0.688
0.714
0.71
0.707
0.705
0.686
0.742
0.701
0.726
0.698
0.707
0.712
0.702
0.697
0.707
0.696
0.701
0.702
0.72
0.728
0.702
0.698
0.691
0.

0.711
0.703
0.689
0.716
0.678
0.695
0.724
0.713
0.674
0.698
0.725
0.688
0.708
0.709
0.714
0.694
0.712
0.706
0.692
0.713
0.67
0.704
0.713
0.695
0.698
0.712
0.715
0.684
0.697
0.708
0.711
0.712
0.682
0.709
0.72
0.714
0.704
0.715
0.687
0.688
0.702
0.704
0.691
0.686
0.731
0.716
0.685
0.704
0.691
0.671
0.676
0.698
0.698
0.716
0.705
0.673
0.697
0.712
0.677
0.689
0.712
0.693
0.699
0.707
0.72
0.704
0.705
0.692
0.732
0.684
0.715
0.697
0.688
0.695
0.677
0.704
0.717
0.726
0.728
0.704
0.712
0.689
0.72
0.697
0.697
0.705
0.695
0.703
0.699
0.717
0.701
0.703
0.719
0.713
0.683
0.722
0.69
0.708
0.693
0.686
0.724
0.689
0.713
0.708
0.716
0.701
0.709
0.694
0.708
0.707
0.708
0.698
0.707
0.722
0.711
0.7
0.684
0.701
0.733
0.698
0.672
0.726
0.715
0.719
0.694
0.703
0.68
0.719
0.684
0.698
0.711
0.71
0.703
0.695
0.698
0.706
0.698
0.693
0.702
0.719
0.684
0.7
0.689
0.721
0.701
0.723
0.708
0.692
0.689
0.684
0.684
0.696
0.738
0.697
0.721
0.718
0.696
0.702
0.694
0.72
0.708
0.689
0.718
0.71
0.693
0.691
0.709
0.704
0.743

0.71
0.692
0.712
0.7
0.695
0.687
0.71
0.711
0.683
0.719
0.715
0.698
0.684
0.717
0.685
0.706
0.703
0.716
0.693
0.684
0.695
0.716
0.701
0.695
0.705
0.688
0.703
0.689
0.68
0.697
0.702
0.672
0.698
0.704
0.69
0.694
0.724
0.702
0.7
0.695
0.711
0.711
0.703
0.68
0.695
0.701
0.715
0.695
0.704
0.717
0.694
0.68
0.704
0.684
0.7
0.687
0.698
0.712
0.695
0.713
0.687
0.692
0.744
0.702
0.713
0.684
0.673
0.713
0.707
0.719
0.681
0.698
0.725
0.729
0.741
0.702
0.695
0.678
0.696
0.714
0.706
0.712
0.693
0.672
0.715
0.704
0.699
0.694
0.703
0.701
0.68
0.69
0.694
0.733
0.697
0.699
0.692
0.704
0.724
0.693
0.718
0.696
0.706
0.686
0.721
0.694
0.712
0.71
0.715
0.704
0.714
0.719
0.715
0.716
0.698
0.69
0.717
0.713
0.693
0.706
0.695
0.681
0.694
0.686
0.685
0.701
0.709
0.728
0.708
0.696
0.689
0.721
0.687
0.695
0.708
0.687
0.677
0.695
0.689
0.723
0.705
0.72
0.686
0.713
0.678
0.712
0.708
0.698
0.707
0.695
0.713
0.696
0.699
0.73
0.716
0.684
0.737
0.706
0.697
0.682
0.696
0.696
0.712
0.688
0.681
0.694
0.694
0.711
0.714
0.72

KeyboardInterrupt: 

In [15]:
X_test = tfidf_transform.transform(count_vec.transform(tweets_test_pd))
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

/home/ayushi/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: invalid value encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))


In [16]:
print("The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=10000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with TFIDF and batch size=10000 is = 49.30362116991643%


In [ ]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, ngram_range=(1,1))
tfidf_train = vec.fit_transform(tweets_train_pd)
tfidf_train_new = SelectPercentile(chi2, percentile=50).fit_transform(tfidf_train, train_classes)
tfidf_train_new

In [11]:

with parallel_backend('threading', n_jobs=2):
    # Your scikit-learn code here

SyntaxError: unexpected EOF while parsing (<ipython-input-11-e3be00bc68e6>, line 4)